In [ ]:
# Binary Classification의 대표적인 2개의 문제를 한번 구현해 보아요!
# 1. 위스콘신 유방암 데이터
# 2. Titanic

In [18]:
# 위스콘신 유방암 데이터를 가지고 구현해보아요!
# 이 데이터는 sklearn이 제공하는 데이터를 사용할 꺼예요!
# sklearn과 tensorflow를 이용해서 구현해 보아요!

import numpy as np
from sklearn import linear_model   # LogisticRegression()
from sklearn.datasets import load_breast_cancer  # 데이터 로딩하기 위한 함수
from sklearn.model_selection import train_test_split  # 학습데이터와 평가데이터 분리
from sklearn.model_selection import cross_val_score  # cross validation하기 위해 필요

# Raw Data Loading
cancer = load_breast_cancer()
# print(type(cancer))   # <class 'sklearn.utils.Bunch'>
                      # sklearn이 데이터를 표현하기 위해 사용하는 자료구조.
                      # python의 dictionary와 유사한 구조.
# print(cancer)      
# data라는 속성과 target이라는 속성을 가지고 있고
# data라는 속성이 독립변수, target이 종속변수
# print(cancer.data.shape, cancer.target.shape)  # (569, 30) (569,)

# print(np.unique(cancer.target, return_counts=True))  
# array([0, 1]), array([212, 357]
# print(cancer.DESCR)  # 유방암 데이터에 대한 상세 내용!
# :Missing Attribute Values: None
# :Class Distribution: 212 - Malignant(악성), 357 - Benign(정상)

# Data Set
x_data = cancer.data
t_data = cancer.target

# Hold-out validation을 위해서 train과 validation데이터를 분리
train_x_data, test_x_data, train_t_data, test_t_data = \
train_test_split(x_data,
                 t_data,
                 test_size=0.2,
                 random_state=2,
                 stratify=t_data)
# print(train_x_data.shape, train_t_data.shape)  # (455, 30) (455,)
# print(np.unique(train_t_data, return_counts=True)) 
# array([0, 1]), array([170, 285])

# Model 생성
model = linear_model.LogisticRegression()

# K-Fold cross validation
test_score = cross_val_score(model, x_data, t_data, scoring='accuracy', cv=5)
print(test_score)  # [0.92982456 0.93859649 0.95614035 0.9122807  0.95575221]
print(test_score.mean())   # 0.9385188635305075

# Hold-out 방식으로 validation
model.fit(train_x_data, train_t_data)
test_score = model.score(test_x_data, test_t_data)
print(test_score)  # 0.9736842105263158

# sklearn으로 구현해 보았어요!

[0.92982456 0.93859649 0.95614035 0.9122807  0.95575221]
0.9385188635305075
0.9736842105263158


C:\Users\campusseven02\anaconda3\envs\machine_TF15\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
C:\Users\campusseven02\anaconda3\envs\machine_TF15\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-lea

In [20]:
# 위의 데이터를 이용해서 이번에는 Tensorflow를 이용해 구현해 보아요!
import tensorflow as tf

## tensorflow 그래프를 그려보아요!

## placeholder
X = tf.placeholder(shape=[None,30], dtype=tf.float32)
T = tf.placeholder(shape=[None,1], dtype=tf.float32)

# Weight & bias
W = tf.Variable(tf.random.normal([30,1]))
b = tf.Variable(tf.random.normal([1]))

# Hypothesis, model, predict model, Logistic Regression Model
logit = tf.matmul(X,W) + b
H = tf.sigmoid(logit)

# cross entropy(loss function)
loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logit,
                                                              labels=T))

# train 
train = tf.train.GradientDescentOptimizer(learning_rate=1e-4).minimize(loss)

# Session, 초기화
sess = tf.Session()
sess.run(tf.global_variables_initializer())   # 초기화 작업.

# 반복학습
# 전체 데이터를 이용해서 1번 학습 => 1 epoch(에폭)
for step in range(100000):
    _, loss_val = sess.run([train, loss], feed_dict={X: train_x_data,
                                                     T: train_t_data.reshape(-1,1)})
    if step % 10000 == 0:
        print('loss value : {}'.format(loss_val))



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



loss value : 1072.21826171875
loss value : 1.074231743812561
loss value : 0.66689133644104
loss value : 0.5257416367530823
loss value : 0.4586733877658844
loss value : 0.4299987554550171
loss value : 0.4145071804523468
loss value : 0.40369462966918945
loss value : 0.3952553868293762
loss value : 0.3879354000091553


In [21]:
# 정확도(accuracy) 측정

# validation data(test_x_data, test_t_data)를 이용해서 정확도를 측정
predict = tf.cast(H >= 0.5, dtype=tf.float32)   # True -> 1.0
                                                # False -> 0.0 
correct = tf.equal(predict, T)     # True, False, False, True, ...  
accuracy = tf.reduce_mean(tf.cast(correct, dtype=tf.float32)) #   1  , 0   ,  0   ,  1

accuracy_val = sess.run(accuracy, feed_dict={X: test_x_data,
                                             T: test_t_data.reshape(-1,1)})
print('Accuracy : {}'.format(accuracy_val))  # 0.9035087823867798

ERROR! Session/line number was not unique in database. History logging moved to new session 57
Accuracy : 0.9035087823867798


In [ ]:
# 두번째 titanic 예제를 이용해서 Logistic Regression을 구현해 보아요!
# Kaggle에서 데이터셋을 받아서 나온 결과를 Kaggle에 upload해서 우리 모델의
# 정확도를 평가받아 보아요!